<a href="https://www.kaggle.com/code/notsau/nlp-positive-negative-neutral?scriptVersionId=161445405" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd

# EDA

In [2]:
df=pd.read_csv("/kaggle/input/squid-game-netflix-twitter-data/tweets_v8.csv")

In [3]:
df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,source,is_retweet
0,the _ûndër-ratèd niggáh👊🏾,NaN,@ManUtd die hard❤️❤️💪🏿💪🏿\n\n\nYOLO\n\n\nJ'ai b...,2019-09-06 19:24:57+00:00,581,1035,8922,False,2021-10-06 12:05:38+00:00,When life hits and the same time poverty strik...,Twitter for Android,False
1,Best uncle on planet earth,NaN,NaN,2013-05-08 19:35:26+00:00,741,730,8432,False,2021-10-06 12:05:22+00:00,That marble episode of #SquidGame ruined me. 😭😭😭,Twitter for Android,False
2,marcie,NaN,animal crossing. chicken nuggets. baby yoda. s...,2009-02-21 10:31:30+00:00,562,1197,62732,False,2021-10-06 12:05:22+00:00,#Squidgame time,Twitter Web App,False
3,YoMo.Mdp,Any pronouns,Where the heck is the karma\nI'm going on my s...,2021-02-14 13:21:22+00:00,3,277,1341,False,2021-10-06 12:05:04+00:00,//Blood on 1st slide\nI'm joining the squidgam...,Twitter Web App,False
4,Laura Reactions,France,I talk and I make reactions videos about shows...,2018-12-19 20:38:28+00:00,330,152,2278,False,2021-10-06 12:05:00+00:00,"The two first games, players were killed by th...",Twitter Web App,False


In [5]:
df.isnull().sum()

user_name               4
user_location       23870
user_description     5211
user_created            0
user_followers          0
user_friends            0
user_favourites         0
user_verified           0
date                    0
text                    0
source                  0
is_retweet              0
dtype: int64

In [7]:
df.shape

(80019, 12)

# Feature Engineering & Finish

In [9]:
df.columns

Index(['user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'text', 'source', 'is_retweet'],
      dtype='object')

In [10]:
df=df[["user_name","text"]]

In [12]:
df["user_name"]=df["user_name"].notnull
#ismi olanları aldık

In [13]:
#!pip install textblob

In [14]:
from textblob import TextBlob

In [15]:
df["text"]=df["text"].str.lower()
df["text"]=df["text"].str.replace("[^\w\s]","")
#noktalama işaretlerini kaldırır
df["text"]=df["text"].str.replace("\d+","")
#rakamları kaldırır
df["text"]=df["text"].str.replace("\r","").replace("\n","")
#satır başları, boşlukları kaldır.

C:\Users\slymn\AppData\Local\Temp\ipykernel_10896\2542781051.py:2: FutureWarning: The default value of regex will change from True to False in a future version.

  df["text"]=df["text"].str.replace("[^\w\s]","")

C:\Users\slymn\AppData\Local\Temp\ipykernel_10896\2542781051.py:4: FutureWarning: The default value of regex will change from True to False in a future version.

  df["text"]=df["text"].str.replace("\d+","")


In [41]:
import nltk
import re
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
df["text"] = df["text"].apply(clean)
#nlp genel kod

[nltk_data] Downloading package stopwords to

[nltk_data]     C:\Users\slymn\AppData\Roaming\nltk_data...

[nltk_data]   Package stopwords is already up-to-date!


In [42]:
df.head()

,user_name,text
0,<bound method Series.notnull of 0 <boun...,life hit time poverti strike yougong yoo let ...
1,<bound method Series.notnull of 0 <boun...,marbl episod squidgam ruin
2,<bound method Series.notnull of 0 <boun...,squidgam time
3,<bound method Series.notnull of 0 <boun...,blood st slideim join squidgam thing im alread...
4,<bound method Series.notnull of 0 <boun...,two first game player kill mask guy bloodi ni...


In [45]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sentiments = SentimentIntensityAnalyzer()
df["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in df["text"]]
df["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in df["text"]]
df["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in df["text"]]
df = df[["text", "Positive", "Negative", "Neutral"]]

[nltk_data] Downloading package vader_lexicon to

[nltk_data]     C:\Users\slymn\AppData\Roaming\nltk_data...

[nltk_data]   Package vader_lexicon is already up-to-date!


In [46]:
df.head()

,text,Positive,Negative,Neutral
0,life hit time poverti strike yougong yoo let ...,0.161,0.101,0.738
1,marbl episod squidgam ruin,0.000,0.559,0.441
2,squidgam time,0.000,0.000,1.000
3,blood st slideim join squidgam thing im alread...,0.126,0.246,0.629
4,two first game player kill mask guy bloodi ni...,0.000,0.461,0.539
